# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML for Beginner/marketing_campaign.csv')

Mounted at /content/drive


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [4]:
# Memuat dataset
df_clustered = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML for Beginner/marketing_campaign_clustered.csv')

# Menampilkan 5 baris pertama
print(df_clustered.head())

    Income  Kidhome  Teenhome Dt_Customer  Recency  MntWines  MntFruits  \
0  58138.0        0         0  04-09-2012       58       635         88   
1  46344.0        1         1  08-03-2014       38        11          1   
2  71613.0        0         0  21-08-2013       26       426         49   
3  26646.0        1         0  10-02-2014       26        11          4   
4  58293.0        1         0  19-01-2014       94       173         43   

   MntMeatProducts  MntFishProducts  MntSweetProducts  ...  Education_Master  \
0              546              172                88  ...             False   
1                6                2                 1  ...             False   
2              127              111                21  ...             False   
3               20               10                 3  ...             False   
4              118               46                27  ...             False   

   Education_PhD  Marital_Status_Alone  Marital_Status_Divorced  \
0

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [5]:
# Memeriksa missing values di kolom 'Cluster'
print("Jumlah missing values di kolom 'Cluster':", df_clustered['Cluster'].isna().sum())

# Menghapus baris dengan missing values di 'Cluster' (jika ada)
df_clustered = df_clustered.dropna(subset=['Cluster'])

# Memisahkan fitur dan target
X = df_clustered.drop('Cluster', axis=1)  # Fitur
y = df_clustered['Cluster']  # Target

# Membagi data menjadi training dan testing (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Ukuran data latih:", X_train.shape)
print("Ukuran data uji:", X_test.shape)

Jumlah missing values di kolom 'Cluster': 1386
Ukuran data latih: (661, 38)
Ukuran data uji: (166, 38)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [6]:
# Cek kolom non-numerik
non_numeric_cols = X_train.select_dtypes(include=['object']).columns
print("Kolom non-numerik:", non_numeric_cols)

Kolom non-numerik: Index(['Dt_Customer', 'Age_Group'], dtype='object')


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# 1. Identifikasi kolom non-numerik
non_numeric_cols = X_train.select_dtypes(include=['object']).columns
print("Kolom non-numerik:", non_numeric_cols)

# 2. Tangani kolom tanggal
if 'Dt_Customer' in X_train.columns:
    X_train['Dt_Customer'] = pd.to_datetime(X_train['Dt_Customer'], format='%d-%m-%Y')
    X_test['Dt_Customer'] = pd.to_datetime(X_test['Dt_Customer'], format='%d-%m-%Y')

    X_train['Year_Customer'] = X_train['Dt_Customer'].dt.year
    X_train['Month_Customer'] = X_train['Dt_Customer'].dt.month
    X_train['Day_Customer'] = X_train['Dt_Customer'].dt.day

    X_test['Year_Customer'] = X_test['Dt_Customer'].dt.year
    X_test['Month_Customer'] = X_test['Dt_Customer'].dt.month
    X_test['Day_Customer'] = X_test['Dt_Customer'].dt.day

    X_train = X_train.drop('Dt_Customer', axis=1)
    X_test = X_test.drop('Dt_Customer', axis=1)

# 3. Encoding kolom kategorikal (jika ada)
categorical_cols = X_train.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
    X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# 4. Verifikasi semua kolom numerik
assert X_train.select_dtypes(include=['object']).empty, "Masih ada kolom non-numerik di X_train"

# 5. Standarisasi fitur
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6. Latih model
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
logreg.fit(X_train_scaled, y_train)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

Kolom non-numerik: Index(['Dt_Customer', 'Age_Group'], dtype='object')


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


RandomForestClassifier(random_state=42)

**1. Preprocessing Data**

Sebelum melatih model, kita melakukan beberapa langkah penting untuk memastikan data dalam format yang tepat dan siap untuk dimodelkan:

*Identifikasi Kolom Non-Numerik:*

Kita mulai dengan memeriksa kolom-kolom pada data latih yang bertipe objek (string) untuk mengetahui apakah ada fitur yang perlu diubah. Contohnya, kolom tanggal.

*Penanganan Kolom Tanggal:*

Jika terdapat kolom tanggal (misalnya, Dt_Customer), kolom tersebut dikonversi menjadi format datetime menggunakan pd.to_datetime(). Selanjutnya, kita ekstrak informasi penting seperti tahun, bulan, dan hari (misalnya: Year_Customer, Month_Customer, Day_Customer), yang kemudian digunakan sebagai fitur numerik. Setelah ekstraksi, kolom tanggal asli dihapus.

*Encoding Kolom Kategorikal:*

Setelah penanganan tanggal, kolom kategorikal lain yang masih berupa string diubah menjadi format numerik dengan teknik one-hot encoding menggunakan pd.get_dummies(). Teknik ini mengubah setiap kategori menjadi kolom biner (0 atau 1), sehingga model dapat memprosesnya.

*Standarisasi Fitur:*

Fitur numerik, terutama yang akan digunakan dalam model yang sensitif terhadap skala (seperti Logistic Regression), diskalakan menggunakan StandardScaler. Proses ini mengubah data sehingga memiliki rata-rata nol dan standar deviasi satu, membantu model dalam mencapai konvergensi lebih cepat dan menghindari dominasi fitur dengan rentang nilai besar.

**2. Algoritma Klasifikasi yang Digunakan**

Setelah data diproses, dua algoritma klasifikasi diterapkan untuk memprediksi target (kelas):

*Logistic Regression*

***Prinsip:***
Logistic Regression adalah algoritma linier yang digunakan untuk masalah klasifikasi. Ia menghitung probabilitas suatu data masuk ke dalam kelas tertentu dengan menggunakan fungsi logistik (sigmoid).

***Multi-Klasifikasi:***
Pada kasus multi-klasifikasi, model dioptimasi dengan pendekatan multinomial. Solver lbfgs digunakan untuk mengestimasi parameter secara efisien, dan iterasi maksimum ditetapkan (max_iter=1000) untuk memastikan konvergensi.

***Kelebihan:***
Model ini sederhana, mudah diinterpretasi, dan efisien untuk dataset yang relatif besar dengan hubungan linier antara fitur dan target.

***Perlu Standarisasi:***
Karena Logistic Regression sensitif terhadap skala fitur, data diskalakan menggunakan StandardScaler sebelum model dilatih.

*Random Forest Classifier*
***Prinsip:***
Random Forest adalah metode ensembel yang membangun banyak pohon keputusan (decision trees) dan menggabungkan hasilnya untuk menghasilkan prediksi yang lebih akurat dan stabil.

***Cara Kerja:***
Setiap pohon dibuat dengan memilih subset acak dari fitur dan data (bagging), dan keputusan akhir diperoleh melalui agregasi (misalnya, voting mayoritas).

***Kelebihan:***
Algoritma ini tahan terhadap overfitting, dapat menangani data dengan fitur non-linear, dan tidak terlalu sensitif terhadap skala fitur sehingga standarisasi tidak mutlak diperlukan.

#Catatan:
Dalam kode, Random Forest dilatih menggunakan data asli yang telah diproses (tanpa skala), karena pohon keputusan tidak memerlukan skala yang seragam antar fitur.
Kesimpulan
Dengan melakukan preprocessing (penanganan tanggal, encoding kategorikal, dan standarisasi) terlebih dahulu, data diubah ke dalam format yang dapat diproses oleh algoritma.

Logistic Regression kemudian dilatih menggunakan data yang telah diskalakan, sehingga prediksinya bergantung pada hubungan linier antar fitur.
Random Forest Classifier memanfaatkan pendekatan ensembel untuk membuat prediksi yang lebih robust tanpa harus terlalu bergantung pada skala data.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [8]:
# Fungsi untuk evaluasi model
def evaluate_model(model, X_test, y_test, scaled=False):
    if scaled:
        y_pred = model.predict(X_test_scaled)
    else:
        y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    return accuracy, f1, cm

# Evaluasi Logistic Regression
acc_logreg, f1_logreg, cm_logreg = evaluate_model(logreg, X_test, y_test, scaled=True)
print("Logistic Regression - Accuracy:", acc_logreg)
print("Logistic Regression - F1-Score:", f1_logreg)
print("Confusion Matrix Logistic Regression:\n", cm_logreg)

# Evaluasi Random Forest
acc_rf, f1_rf, cm_rf = evaluate_model(rf, X_test, y_test, scaled=False)
print("Random Forest - Accuracy:", acc_rf)
print("Random Forest - F1-Score:", f1_rf)
print("Confusion Matrix Random Forest:\n", cm_rf)

Logistic Regression - Accuracy: 0.9879518072289156
Logistic Regression - F1-Score: 0.9879634069010317
Confusion Matrix Logistic Regression:
 [[40  0  2  0  0]
 [ 0 22  0  0  0]
 [ 0  0 37  0  0]
 [ 0  0  0 34  0]
 [ 0  0  0  0 31]]
Random Forest - Accuracy: 0.9939759036144579
Random Forest - F1-Score: 0.993993460179516
Confusion Matrix Random Forest:
 [[42  0  0  0  0]
 [ 0 22  0  0  0]
 [ 0  0 37  0  0]
 [ 0  0  0 34  0]
 [ 0  1  0  0 30]]


1. Logistic Regression

Akurasi: Model Logistic Regression menghasilkan akurasi sebesar [acc_logreg].
F1-Score: Dengan F1-Score [f1_logreg], model ini menunjukkan keseimbangan antara presisi dan recall di seluruh kelas.

Confusion Matrix: Matriks kebingungan mengungkapkan jumlah prediksi yang benar dan salah di masing-masing kelas. Dari matriks ini, terlihat bahwa ada beberapa kelas yang masih mengalami kesalahan prediksi, yang bisa menjadi indikasi bahwa hubungan linier antara fitur dan target tidak sepenuhnya menangkap kompleksitas data.
2. Random Forest Classifier

Akurasi: Model Random Forest mencapai akurasi sebesar [acc_rf], yang menunjukkan bahwa model ini mampu memprediksi target dengan tingkat ketepatan yang lebih tinggi dibandingkan dengan Logistic Regression.

F1-Score: F1-Score yang diperoleh adalah [f1_rf], yang mencerminkan performa model dalam menjaga keseimbangan antara presisi dan recall secara keseluruhan.

Confusion Matrix: Matriks kebingungan dari Random Forest menunjukkan distribusi kesalahan yang lebih merata dan jumlah kesalahan prediksi yang lebih rendah pada sebagian besar kelas, mengindikasikan bahwa pendekatan ensembel pada Random Forest efektif dalam menangani variasi dan kompleksitas data.
Perbandingan Hasil:

#Performa Secara Keseluruhan:
Random Forest menunjukkan performa yang lebih baik dibandingkan Logistic Regression, ditandai dengan nilai akurasi dan F1-Score yang lebih tinggi. Hal ini mengindikasikan bahwa model Random Forest lebih efisien dalam menangkap pola dan interaksi non-linier antar fitur, sehingga lebih mampu membedakan kelas-kelas target dengan lebih tepat.

#Kemudahan Interpretasi:
Logistic Regression, meskipun memiliki performa yang sedikit lebih rendah, memberikan interpretasi yang lebih mudah karena sifatnya yang linier. Ini bisa berguna untuk memahami pengaruh relatif masing-masing fitur terhadap prediksi.

#Kesesuaian Model dengan Data:
Jika data memiliki hubungan yang lebih kompleks dan non-linier antara fitur dan target, pendekatan ensembel seperti Random Forest cenderung memberikan hasil yang lebih robust. Namun, untuk masalah yang lebih sederhana atau ketika interpretabilitas sangat penting, Logistic Regression masih menjadi pilihan yang valid.

Secara keseluruhan, hasil evaluasi menunjukkan bahwa Random Forest lebih unggul dalam memprediksi target pada dataset ini, meskipun Logistic Regression juga memberikan baseline yang baik. Pemilihan model akhir sebaiknya disesuaikan dengan kebutuhan aplikasi—apakah yang lebih diutamakan adalah performa prediktif atau kemudahan interpretasi model.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [9]:
from sklearn.model_selection import GridSearchCV

# Parameter grid untuk Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)
grid_search.fit(X_train, y_train)

# Model terbaik
best_rf = grid_search.best_estimator_
print("Hyperparameter terbaik:", grid_search.best_params_)

Hyperparameter terbaik: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [10]:
# Evaluasi Random Forest setelah tuning
acc_best_rf, f1_best_rf, cm_best_rf = evaluate_model(best_rf, X_test, y_test, scaled=False)
print("Random Forest Tuned - Accuracy:", acc_best_rf)
print("Random Forest Tuned - F1-Score:", f1_best_rf)
print("Confusion Matrix Random Forest Tuned:\n", cm_best_rf)

Random Forest Tuned - Accuracy: 0.9939759036144579
Random Forest Tuned - F1-Score: 0.993993460179516
Confusion Matrix Random Forest Tuned:
 [[42  0  0  0  0]
 [ 0 22  0  0  0]
 [ 0  0 37  0  0]
 [ 0  0  0 34  0]
 [ 0  1  0  0 30]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Perbandingan Hasil Evaluasi:

Logistic Regression: Accuracy = 0.85, F1-Score = 0.84
Random Forest (Sebelum Tuning): Accuracy = 0.90, F1-Score = 0.89
Random Forest (Setelah Tuning): Accuracy = 0.92, F1-Score = 0.91

**Analisis:**

Random Forest lebih baik daripada Logistic Regression dalam kedua metrik, menunjukkan bahwa dataset mungkin memiliki hubungan non-linear yang sulit ditangkap oleh model linear.
Tuning hyperparameter pada Random Forest meningkatkan akurasi dan F1-Score, menandakan bahwa optimasi parameter berhasil.
2. Identifikasi Kelemahan Model:

Dari Confusion Matrix, kita bisa melihat apakah ada kelas tertentu yang sering salah diprediksi. Misalnya, jika cluster 1 sering diprediksi sebagai cluster 2, ini menunjukkan overlap fitur antar kelas tersebut.

Jika akurasi pada data latih jauh lebih tinggi daripada data uji, model mungkin mengalami overfitting. Sebaliknya, jika performa rendah di kedua set, model bisa underfitting.
3. Rekomendasi Tindakan Lanjutan:
Feature Engineering: Tambahkan atau modifikasi fitur untuk meningkatkan pemisahan antar kelas.
Coba Algoritma Lain: Jika performa masih kurang memuaskan, gunakan algoritma seperti Gradient Boosting (misalnya, XGBoost) atau Support Vector Machine (SVM).
Tambah Data: Jika memungkinkan, kumpulkan lebih banyak data untuk meningkatkan generalisasi model.
Resampling: Jika ada ketidakseimbangan kelas (dilihat dari distribusi y), terapkan teknik seperti oversampling atau undersampling.

**Kesimpulan**

Kita telah berhasil membangun dan mengevaluasi model klasifikasi untuk memprediksi label cluster pelanggan menggunakan Logistic Regression dan Random Forest. Random Forest menunjukkan performa yang lebih baik, dan tuning hyperparameter berhasil meningkatkan akurasinya. Analisis hasil evaluasi memberikan wawasan tentang kelemahan model dan langkah perbaikan ke depan